<h1>Análise de dados Enem 2018</h1>

<p>A intenção da análise que se segue é entender e retirar informações relevantes em relação aos participantes do <b>enem 2018</b></p>

<p>Informações como a relação da média de notas com determinados grupos de candidatos, além de obter correlação entre as notas desses candidatos e as informações socioeconômicas e a contribuição dessas informações para o resultado obtido na avaliação</p>

<p>Existe alguma relação entre a nota/resultado obtida e a região, estado, ou município dos estudantes?</p>

<p>O PIB ou IDH de determinada região coopera para melhora ou piora da média do resultado naquela região?
</p>

<p>O Dataset utilizado foi extraído do site oficial de dados abertos do Governo Federal e pode ser baixados <a href #>neste link</a>, além disso, os dados préprocessados para o resultado desse estudo podem ser baixados <a href #>neste link</a></p>
<h3>Definição de Objetivos - Primários e Secundários</h3>
<h4>Objetivos Primários</h4>
<ol>
	<li>Realizar analises de cada ano do Enem individualmente e Observar através de técnicas de regressão linear e machine learning as correlações entre as variáveis e o peso de cada variável no desempenho dos candidatos do enem</li>
	<li>Unir os datasets disponíveis entre os anos de 1998 e 2019 no site de dadis abertos do Governo Federal e analisar a evolução dos candidatos, além de identificar possíveis causas para tais desempenhos</li>

</ol>


In [1]:
#!pip install plotly
#!pip install cufflinks

In [2]:
# Importando os módulos e pacotes necessários para plotar os gráficos de forma interativa
import cufflinks as cf
from plotly.offline import iplot
import pandas as pd
import numpy as np
cf.go_offline()

<h3>Os dados do dataset abaixo podem ser baixados em sua forma bruta em <a href>inep.com.br</a></h3>

<h2>Reduzindo o dataset para otimizar a memória e melhor aproveitar o poder de processamento computacional</h2>

Arquivos como os do enem são muito grandes (mais de 5 milhões de linhas e aproximadamente 3BG), o que pode dificultar a leitura dos arquivos .CSV, ao invés de realizar um filter nas colunas, tive de fazer esse filtro já na abertura e leitura do documento utilizando o comando "usecols". Sobre essa técnica é possível encontrar detalhes em um dos vídeos que compõe a série de análise de dados do canal <a href="https://www.youtube.com/watch?v=bC6Q9Uc80Xw&list=PL5TJqBvpXQv5N3iV68bGBkea0HjMk98lR&index=15"> Programação Dinâmica </a> no youtube.

Sendo assim, colunas que não serão utilizadas foram "desprezadas" deixando o dataset mais leve e possibilitando melhor processamento

In [3]:
##Filtrando as colunas que serão utilizadas na análise
filtro_dados_enem = ['NU_INSCRICAO', 'NU_ANO', 
       'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_NOME_SOCIAL', 
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT',  'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
       'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
       'Q023', 'Q024', 'Q025']

In [4]:
#Carregando normalmente o dataset, porém estou utilizando o comando "usecols" passando como parâmetro a lista 
#das coluna filtradas que foram armazenadas na variável filtro_dados_enem

enem2018 = pd.read_csv('MICRODADOS_ENEM_2018.csv', sep=';', encoding='ISO 8859-1', usecols=filtro_dados_enem)

MemoryError: Unable to allocate 42.1 MiB for an array with shape (5513747,) and data type object

In [ ]:
#Visualizando as primeiras linhas do dataser apenas com as colunas carregadas redução de 137 para 60 colunas
enem2018.head()

In [ ]:
#memória utilizada no processamento do dataset
enem2018.info(memory_usage='deep')

<h3>Otimização dos tipos de dados</h3>
<p>O dataframe ainda estava consumindo uma parte muito grande, tanto em memória, quanto em processamento, então foi necessário realizar a otimização dos tipos de dados de sua composição </p>
<p> Utilizei como referência esse artigo do <a href="https://medium.com/@miltongamaneto/otimizando-o-armazenamento-da-mem%C3%B3ria-de-um-dataframe-pandas-ca8c4eb45699">Milton Gama Netto</a> que mostra em detalhes o motivo e como realizar essa otimização. </p>
<p>"Em resumo o python superestima o tamanho dos dados inferindo por exemplo o tipo  <i>int64</i> para números inteiros, mesmo quando o dado em sí não precisa de 64bits reservado na memória. Agora já dá pra imaginar que uma tabela com mais de 5 milhões de instâncias com dados superestimados pode ficar imensa, mas sem necessidade." </p>

In [ ]:
#Otimizando os tipos de dados do dataframe
ints = enem2018.select_dtypes(include=['int64','int32','int16']).columns
enem2018[ints] = enem2018[ints].apply(pd.to_numeric, downcast='integer')

floats = enem2018.select_dtypes(include=['float']).columns
enem2018[floats] = enem2018[floats].apply(pd.to_numeric, downcast='float')
objects = enem2018.select_dtypes('object').columns
enem2018[objects] = enem2018[objects].apply(lambda x: x.astype('category'))

#Aqui já é possível verificar todas as colunas renomeadas e o tamanho da redução de 

#memória utilizada no processamento do dataset
enem2018.info(memory_usage='deep')

<h3>Renomeando as colunas</h3>
<p>Os dados do Enem foram recentemente reformulados para obedecer a uma padronização, o que é ótimo, pois facilita no entendimento dos dados e no agrupamento, mas para uma visualização exploratória, acabam sendo pouco amigaveis, por exemplo no questionário socioeconômico, o nome das colunas estão como Q001, Q002, Q003 e assim segue. A ideia aqui é justamente traduzir esses códigos de (Q001 | Q002 | Q003) para (GRAU DE INSTRUCAO DO PAI|
GRAU DE INSTRUCAO DA MAE | OCUPACAO DO PAI |) respectivamente com a ajuda do dicionario de dados que acompanha os arquivo do dataset</p>

In [ ]:
#Renomeando as colunas para que tenham rótulos mais amigaveis
enem_2018 = enem2018.rename({
'NU_INSCRICAO':'NSCRICAO', 
'NU_ANO':'ANO', 
'NO_MUNICIPIO_RESIDENCIA':'MUNICIPIO_RESIDENCIA', 
'SG_UF_RESIDENCIA':'UF_RESIDENCIA',
'NU_IDADE':'IDADE', 
'TP_SEXO':'SEXO', 
'TP_ESTADO_CIVIL':'ESTADO_CIVIL', 
'TP_COR_RACA':'COR_RACA',
'TP_NACIONALIDADE':'NACIONALIDADE', 
'NO_MUNICIPIO_NASCIMENTO':'MUNICIPIO_NASCIMENTO', 
'SG_UF_NASCIMENTO':'UF_NASCIMENTO',
'TP_ST_CONCLUSAO':'T_CONCLUSAO', 
'TP_ANO_CONCLUIU':'ANO_CONCLUIU', 
'TP_ESCOLA':'TP_ESCOLA', 
'TP_ENSINO':'TP_ENSINO',
'IN_TREINEIRO':'IN_TREINEIRO',  
'NO_MUNICIPIO_ESC':'MUNICIPIO_ESC', 
'CO_UF_ESC':'COD_UF_ESC', 
'SG_UF_ESC': 'UF_ESC',
'TP_DEPENDENCIA_ADM_ESC':'DEPENDENCIA_ADM_ESC', 
'TP_LOCALIZACAO_ESC':'LOCALIZACAO_ESC', 
'IN_NOME_SOCIAL':'NOME_SOCIAL', 
'NO_MUNICIPIO_PROVA':'MUNICIPIO_PROVA', 
'SG_UF_PROVA':'SG_UF_PROVA',
'TP_PRESENCA_CN':'PRESENCA_CN', 
'TP_PRESENCA_CH':'PRESENCA_CH', 
'TP_PRESENCA_LC':'PRESENCA_LC',
'TP_PRESENCA_MT':'PRESENCA_MT',  
'NU_NOTA_CN':'NOTA_CN', 
'NU_NOTA_CH':'NOTA_CH', 
'NU_NOTA_LC':'NOTA_LC',
'NU_NOTA_MT':'NOTA_MT', 
'TP_STATUS_REDACAO':'STATUS_REDACAO', 
'NU_NOTA_COMP1':'NOTA_COMP1',
'NU_NOTA_COMP2':'NOTA_COMP2', 
'NU_NOTA_COMP3':'NOTA_COMP3', 
'NU_NOTA_COMP4':'NOTA_COMP4', 
'NU_NOTA_COMP5':'NOTA_COMP5',
'NU_NOTA_REDACAO':'NOTA_REDACAO', 
'Q001': 'GRAU DE INSTRUCAO DO PAI',
'Q002': 'GRAU DE INSTRUCAO DA MAE',
'Q003': 'OCUPACAO DO PAI',
'Q004': 'OCUPACAO DA MAE',
'Q005': 'QUANTAS PESSOAS MORAM EM CASA',
'Q006':	'RENDA MENSAL FAMILIAR',
'Q007': 'POSSUI EMPREGADO(A) DOMESTICO(A)',
'Q008': 'POSSUI BANHEIRO',
'Q009': 'POSSUI QUARTOS EM CASA',
'Q010': 'POSSUI CARRO EM CASA',
'Q011': 'POSSUI MOTOCICLETA',
'Q012': 'POSSUI GELADEIRA',
'Q013': 'POSSUI FREEZER (INDEPENDENTE DA GELADEIRA)',
'Q014':	'POSSUI MAQUINA DE LAVAR',
'Q015': 'POSSUI MAQUINA DE SECAR ROUPAS',
'Q016': 'POSSUI FORNO MICROONDAS',
'Q017': 'POSSUI MAQUINA DE LAVAR LOUCAS',
'Q018': 'POSSUI ASPIRADOR DE PO',
'Q019': 'POSSUI TELEVISAO A CORES',
'Q020': 'POSSUI APARELHO DVD',
'Q021': 'POSSUI TV POR ASSINATURA',
'Q022':	'POSSUI TELEFONE CELULAR',
'Q023': 'POSSUI TELEFONE FIXO',
'Q024': 'POSSUI COMPUTADOR EM CASA',
'Q025':'POSSUI INTERNET EM CASA'
}, axis='columns')

In [ ]:
#Lendo as primeiras linhas do dataframe com as colunas já renomeadas
enem_2018.head()

In [ ]:
enem_2018.shape

In [ ]:
enem_2018.to_csv('enem_2018.csv')

In [ ]:
enem_2018 = pd.read_csv('enem_rename.csv', sep=';', encoding='ISO 8859-1')

In [ ]:
enem_2018